In [1]:
%cd /data/joohyun7u/project/vq2d-lightning
%load_ext autoreload
%autoreload 2

/data/joohyun7u/project/vq2d-lightning


/data/joohyun7u/anaconda3/envs/vqlight3/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from IPython.display import Image as IPImage

from ltvu.lit.model import LitModule
from ltvu.lit.data import LitVQ2DDataModule

import hydra
from hydra.core.global_hydra import GlobalHydra
from omegaconf import OmegaConf
# from diffusers.utils import make_image_grid

import torch
import torchvision.transforms.functional as TF
from einops import rearrange, repeat
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO


def ten2pil(tensor, pad: float = 0.02, alpha = None, cmap = 'viridis'):
    assert tensor.dim() in (2, 3)  # (H, W), (C, H, W)
    tensor = tensor.cpu()
    tensor -= tensor.min()
    tensor /= tensor.max()

    if tensor.dim() == 3:
        tensor = tensor.permute(1, 2, 0)
        assert tensor.shape[-1] == 3

    fig = plt.figure(figsize=(5, 5))
    ax = plt.Axes(fig, [pad, pad, 1. - 2* pad, 1. - 2 * pad])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(tensor, aspect='equal', alpha=alpha, cmap=cmap)

    plots_io = BytesIO()
    fig.savefig(plots_io, format='jpg' if alpha is None else 'png', bbox_inches='tight', pad_inches=0)
    plt.close()

    img = Image.open(plots_io)
    # plots_io.close()
    return img

GlobalHydra.instance().clear()
OmegaConf.clear_resolvers()
OmegaConf.register_new_resolver("job_type", lambda : 'debug')
OmegaConf.register_new_resolver('runtime_outdir', lambda : 'outputs/tmp')
OmegaConf.register_new_resolver("eval", eval)
OmegaConf.register_new_resolver("tuple", lambda *args: tuple(args))
hydra.initialize(config_path='config', job_name='asdasdasdasdasd')

/data/joohyun7u/anaconda3/envs/vqlight3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_577641/2392022928.py:51: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(config_path='config', job_name='asdasdasdasdasd')


hydra.initialize()

In [3]:
import os
os.environ["SLURM_GPUS_ON_NODE"] = "1"  # 기본값 설정

In [4]:
path_ckpt = '../outputs/batch/2024-10-22/54448/epoch=78-iou=0.5046.ckpt'
plm = LitModule.load_from_checkpoint(path_ckpt).cuda()

eval_config = hydra.compose(config_name='base_jh', overrides=[
    # f'ckpt={path_ckpt.replace('=', '\\=')}',
    f'batch_size=1',
    f'num_workers=4',
    f'prefetch_factor=1'
])
pdm = LitVQ2DDataModule(eval_config)  # won't use trainer here nor batched forward pass so no need to load the eval config and plm.config is enough
pdm.batch_size = 1

In [5]:
import kornia
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
normalization = kornia.enhance.Normalize(mean=MEAN, std=STD)

In [ ]:
import torch._dynamo
import json
torch._dynamo.config.suppress_errors = True

from pathlib import Path
# for batch in pdm.train_dataloader():

p_ann = Path('../data/vq_v2_train_anno.json')
all_anns = json.load(p_ann.open())

b,bb, bidx= 0, 0, 0
# for bidx in range(13607):
while True:
    if bidx == 13607:
        break
    rt_ann = [ann['fno'] for ann in all_anns[bidx]['response_track']]
    anns = all_anns[bidx]
    bb += len(rt_ann)
    a = 0
    clip_uid, annotation_uid, query_set = anns['clip_uid'], anns['annotation_uid'], anns['query_set']
    for idx, frame_idx in enumerate(rt_ann):
        file_path = Path('../outputs/rt_pos_quries_feat') / 'train' / clip_uid / f'{clip_uid}_{frame_idx}_{annotation_uid}_{query_set}.pt'
        if not file_path.exists():
            print(file_path)
            a +=1
        else:
            b+= 1
    if a == 0:
        bidx += 1
        continue
    print(bidx, a, 'rt_len :', len(rt_ann))
    
    batch = pdm.get_train_sample(idx=bidx)
    print(rt_ann[0],rt_ann[-1], '::', batch['experiment']['multi_query']['rt_pos_idx'])
    rt_pos_queries = batch['experiment']['multi_query']['rt_pos_queries']  # [b, #Q, c, h, w]
    bsz = rt_pos_queries.shape[0]
    rt_pos_queries = rearrange(rt_pos_queries, 'b q c h w -> (b q) c h w')
    rt_pos_queries = normalization(rt_pos_queries)  # [b*#Q, c, h, w]
    rt_pos_queries = rearrange(rt_pos_queries, '(b q) c h w -> b q c h w', b=bsz)
    batch['rt_pos_queries'] = rt_pos_queries
    for k, v in batch.items():
        if isinstance(v, torch.Tensor):
            batch[k] = v.cuda().float() #.to(torch.bfloat16)
         
    extra_args = {}
    extra_args['sim_mode']='max'
    extra_args['sim_thr']=0.0
    extra_args['enable_rt_pq_threshold']= False
    extra_args['rt_pos']=True
    extra_args['get_intermediate_features']=True
    plm.model = plm.model.cuda().to(torch.bfloat16)
    output_dict, output_query = plm.model.extract_rt_feat(**batch, compute_loss=True, **extra_args)
    for b, (clip_uid, frame_idxs, annotation_uid, query_set) in enumerate(zip(batch['clip_uid'], batch['experiment']['multi_query']['rt_pos_idx'], batch['annotation_uid'], batch['query_set'])):
        for idx, frame_idx in enumerate(frame_idxs):
            if frame_idx != -1:
                p_feat_out_dir = Path('../outputs/rt_pos_quries_feat') / 'train' / clip_uid
                p_feat_out_dir.mkdir(parents=True, exist_ok=True)
                file_path = p_feat_out_dir / f'{clip_uid}_{frame_idx}_{annotation_uid}_{query_set}.pt'
                if file_path.exists():
                    pass
                else:
                    print(file_path)
                    torch.save(output_dict[b,idx,:].detach().cpu(), file_path)
        
print('total' , b, bb)


In [9]:
batch['experiment']['multi_query']['rt_pos_idx']

tensor([[910, 911, 912, 913, 914, 915, 916, 917, 918, 919, 920, 921, 922, 923,
         924, 925, 926, 927, 928, 929, 930, 931, 932, 933, 934, 935, 936, 937,
         938, 939, 940, 941]])

In [8]:
import json


from pathlib import Path
p_ann = Path('/data/joohyun7u/project/vq2d-lightning/data/vq_v2_train_anno.json')
all_anns = json.load(p_ann.open())

b,bb, bidx= 0, 0, 0
# for bidx in range(13607):

rt_ann = [ann['fno'] for ann in all_anns[bidx]['response_track']]
anns = all_anns[bidx]
anns['response_track_valid_range']

[1099, 1109]

In [ ]:
import os
import torch

# 대상 디렉터리 설정
root_dir = '/data/joohyun7u/project/vq2d-lightning/outputs/rt_pos_quries_feat/train'
i = 0
# 모든 하위 디렉터리를 탐색하며 .pt 파일 처리
for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.pt'):
            file_path = os.path.join(subdir, file)
            
            # .pt 파일 로드, detach 후 CPU로 이동
            tensor = torch.load(file_path, map_location='cpu').detach().cpu()
            
            # 텐서를 같은 경로에 다시 저장
            torch.save(tensor, file_path)
            print(f"{i} Processed and saved: {file_path}")
            i+=1
